In [1]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

# -1) Cycliste

In [2]:
class logger():
    """
    Save data into memory (in an array) and write it down to a file after [nb_log] is reached.
    """
    def __init__(self, path, limit_to_save_to_disk=100):
        self.path    = path
        self.limit   = limit_to_save_to_disk
        self.logs    = []
        self.nb_logs = 0
        
    def log(self, string):
        self.logs.append(string)
        self.nb_logs = self.nb_logs +1
        if self.limit < self.nb_logs:
            self.write_to_disk()
    
    def write_to_disk(self):
        with open(self.path, "a") as f:
            while len(self.logs):
                try:
                    a = self.logs.pop()
                    f.write(u"%s\n"%a.decode('ascii', errors='ignore'))
                except:
                    print "pbm with "
                    print a
        self.nb_logs = len(self.logs)
    def __del__(self):
        self.write_to_disk()
        

In [3]:
!git status


On branch master
Your branch is up-to-date with 'origin/master'.
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   velib2.ipynb

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	deleted:    logs/.keep

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	logs/cycliste_cyclistes.csv
	logs/cycliste_debug.csv
	logs/cycliste_prises.csv
	logs/cycliste_rendu.csv
	logs/stations_dispos.csv
	logs_backup/



In [4]:
!git add scala_1.ipynb velib2.ipynb

In [221]:
class Cycliste:
    def test(self):
        print get_nearest_station(1,1)
    def __init__(self, parameters):
        import time
        from scipy.stats import norm
        import random
        self.station_maison  = parameters[1]
        self.station_travail = parameters[2]
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        if age < 35:
            facteur = norm( loc =1.2, scale=0.2 )
        elif age <50:
            facteur = norm( loc =1  , scale=0.2 )
        elif age <90:
            facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base = 20
            nb_km = norm( loc =7, scale=2 )
        else:
            vitesse_de_base = 15
            nb_km = norm( loc =10, scale=2 )
        vitesse_moyenne = vitesse_de_base * facteur.rvs(1) + sportif
        
        self.sportif     = sportif
        self.age         = age
        self.sexe        = sexe
        self.nb_km       = nb_km.rvs(1)[0]
        self.vitesse     = vitesse_moyenne[0]
        self.sur_velo    = False
        self.debug       = False
        self.nb_trajet   = 0
        self.durees      = []
        self.trajets     = []
        self.velo        = False
        self.identifiant = "cycliste_%s"%parameters[0]
        self.prises      = []
        self.attente         = random.randint(0,100)
        self.last_rendu      = time.time()
        self.logger_cycliste = logger("./logs/cycliste_cyclistes.csv")
        self.logger_prise    = logger("./logs/cycliste_prises.csv")
        self.logger_rendu    = logger("./logs/cycliste_rendu.csv")
        self.logger_debug    = logger("./logs/cycliste_debug.csv")
        self.a_la_maison = True
        self.log_cycliste()
        
    def log_debug(self, message):
        from time import time
        to_print = u"%s,%s,%s"% (self.identifiant, time() , message)
        self.logger_debug.log(to_print)
    
    def peut_prendre_velo(self, station):
        """
        Renvoie un velo ou bien False.
        """
        from time import time
        if time() < self.attente + self.last_rendu:
            self.log_debug(u"trop tot")
            return False
        if self.sur_velo:
            self.log_debug(u"deja sur velo")
            return False
        for local_station in station.nearest:
            velo = local_station.prendre_velo()
            if not velo:
                self.log_debug(u"pas de velo sur la station %s " % local_station)
            else:
                return velo 
        return False
        
    def prend_velo(self, station):
        import time
        if self.a_la_maison:
            station = self.station_maison
        else:
            station = self.station_travail
        
        velo = self.peut_prendre_velo(station)
        if not velo :
            return
        
        velo.disponible      = False
        self.sur_velo        = True
        self.heure_de_depart = time.time()
        self.velo            = velo
        self.log_prise()
        try:
            nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
        except Exception as e:
            self.log_debug(u"pbm sur nb_km_local = self.nb_km = %s " %self.nb_km)
            nb_km_local = 15
        try:
            vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
            vitesse_local        = vitesse_local * velo.performance
        except:
            self.log_debug(u"pbm sur vitesse_local self.vitesse = %s"%self.vitesse)
            vitesse_local = self.vitesse
            
        self.nb_km_trajet    = nb_km_local
        duree                = nb_km_local / vitesse_local
        self.heure_d_arrivee = self.heure_de_depart + duree
        
        if velo.performance <0.1:
            duree                = 0
            self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart
            self.nb_km_trajet    = 0
            self.rendre_velo(station)
        elif velo.performance <0.3:
            duree                = duree / 10
            self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart + duree
            self.nb_km_trajet    = self.nb_km_trajet / float(10)
            self.rendre_velo(station)
        elif velo.performance <0.5:
            duree                = duree / 5
            self.durees.append( duree)
            self.nb_km_trajet    = self.nb_km_trajet / float(5)
            self.heure_d_arrivee =  self.heure_de_depart + duree 
        else:
            self.durees.append( duree)




    def rendre_velo(self, station):        
        
        import time
        if not self.sur_velo:
            if self.debug : print u"pas en chemin",
            return 0
        if time.time() > self.heure_d_arrivee:
            if station.rendre_velo(self.velo, self.nb_km_trajet):
                
                #if self.debug : print "plus sur velo",
                duree_constatee = self.heure_d_arrivee - self.heure_de_depart
                trajet          = (self.heure_de_depart, self.heure_d_arrivee, self.identifiant, self.nb_km_trajet, self.velo.nom )
                self.log_rendu()
                self.a_la_maison = False
                self.last_rendu = time.time()
                self.sur_velo  = False
                self.nb_trajet = self.nb_trajet + 1
                self.velo      = False
                self.trajets.append(trajet)
                
            else:
                if self.debug : print u"impossible de rendre sur station %s"%(station),
                pass
        else:
            if self.debug : print u"pas encore arrive",
            pass
        
    def log_cycliste(self):
        to_print = u"%s,%s,%s,%s,%s,%s,%s"%(    self.identifiant   ,self.sportif     ,
                                                 self.age         ,self.sexe        ,
                                                 self.nb_km       ,self.vitesse     , self.attente )
        
        self.logger_cycliste.log(to_print)
            
    def log_prise(self, action="prise"):
        to_print = u"%s,%s,%s,%s"%( self.identifiant,self.heure_de_depart, self.velo.nom, action)
        self.logger_prise.log(to_print)
            
    def log_rendu(self, action="prise"):
        to_print = u"%s,%s,%s,%s,%s\n"%( self.identifiant, self.heure_de_depart, 
                                       self.heure_d_arrivee,  self.nb_km_trajet, self.velo.nom )

        self.logger_rendu.log(to_print)


# -1) Velo

In [222]:
class Velo:

    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        self.nom          = nom
        performance       = norm( loc =0.9, scale=0.2 )
        self.performance  = performance.rvs(1)[0]
        self.station      = station
        self.degradation  = 0
        self.performances = []
        self.debug        = False
        self.nb_km_trajet = 0
        self.heures_rendu = []
        self.disponible   = True
        self.logger_reparation = logger("./logs/velo_reparations.csv")
        self.logger_etat       = logger("./logs/velos_etats.csv")
        self.probleme_list     = self.get_problemes_list()

    def get_problemes_list(self):
        """
        Créé le tableau de dégradations possible pour un vélo.
        """
        problemes       = [0] * 50
        problemes_bis   = {u"pedale" : 0.2, u"roue"  : 1 , u"degonfle" : 0.3, u"freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.values())
        return problemes
        
    def log_reparation(self):
        import time
        reparation =(self.nom,time.time(),self.performance)
        to_print = u"%s,%s,%s"%reparation
        self.logger_reparation.log(to_print)
        

    def log_etat(self):
        from time import time
        to_print = u"%s,%s,%s,%s,%s\n"%(   self.nom           ,
                                           time()            ,  
                                           self.station.nom  ,
                                           self.performance  ,
                                           self.nb_km_trajet )
        self.logger_etat.log(to_print)
    
    def rendu(self, station, nb_km_trajet):
        import time
        self.disponible = True
        if self.debug : print(u"velo %s rendu sur station %s"%(self, station))
            
        degradation       = random.choice(self.probleme_list)        
        self.performance  = self.performance - self.performance*degradation                                        
        self.nb_km_trajet = self.nb_km_trajet + nb_km_trajet
        self.station = station
        self.log_etat()
        if 900 < random.randint(0, 1000 ) :
            self.performance  = 1
            self.log_reparation()
        
        

    def __str__(self):
        return "%s"%self.nom



# -1 ) Station

In [223]:
class Station:
    def __init__(self, parameters, debug=False ):
        
        import random
        import time
        debut         = time.time()
        self.position = parameters[1]
        self.nom      = parameters[0]
        self.nb_plot  = random.choice([10,15,20, 30])
        self.nb_libre = 0 #random.randint(0, self.nb_plot )
        self.nb_velos = self.nb_plot # - self.nb_libre
        self.velos    = []
        noms          = get_names()
        
        for n in range(self.nb_velos ):
            nom_velo= u"velo_"+"".join(["%s"%x for x in random.choice(noms)])
            self.velos.append(Velo(nom_velo , self))
        
        if debug : print u"created %s velos"%len(self.velos),
        self.debug          = False
        self.avaries        = []
        self.arrete         = False
        duree               = time.time() - debut
        if debug : print u"station créée en %s"%duree
        
        self.logger_dispos = logger("./logs/stations_dispos.csv")
        self.logger_reparation = logger("./logs/stations_reparations.csv")
        self.logger_avarie = logger("./logs/stations_avarie.csv")
        
        self.liste_d_avaries = self.get_avarie_list()
        self.log_dispo()



    def log_reparation(self):
        from time import time
        dispo =(self.nom, time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s"%dispo
        self.logger_reparation.log(to_print)
            
            
    def log_dispo(self):
        from time import time
        dispo =(self.nom,time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s"%dispo
        self.logger_dispos.log(to_print)
            
    def log_avarie(self, avarie):
        from time import time
        to_print = u"%s,%s,%s"%(self.nom, time(), avarie )
        self.logger_avarie.log(to_print)

        
    def get_avarie_list(self):
        avaries     = [0]*100000
        plot        = u"plot mort"
        half        = u"50% mort"
        network     = u"reseau"
        electricite = u"eteint"
        bug         = u"bug"
        problemes   = { plot       : 20 ,
                       half        : 10 ,
                       network     :  5 ,
                       electricite : 30 , 
                       bug         :  3 }
        
        for k,v in problemes.iteritems():
            avaries.extend([k] * v)
        return avaries
    
    def avarie_possible(self):
        import math
        import random
        import time
        
        plot        = u"plot mort"
        half        = u"50% mort"
        network     = u"reseau"
        electricite = u"eteint"
        bug         = u"bug"
        
        avaries = list(self.liste_d_avaries)
        if plot in self.avaries : avaries.extend( [plot] * 60 )
        if half in self.avaries : avaries.extend( [half] *100 )
        
        avarie = random.choice(avaries)
        self.avaries.extend([avarie])
        if avarie == plot :
            self.nb_libre = self.nb_libre -1
        elif avarie == half:
            self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
        elif avarie == network :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == electricite :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == bug :
            if self.nb_libre >0:
                self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
        if self.nb_libre <0:
            self.nb_libre = 0            
        self.log_dispo()
        
        if avarie:
            self.log_avarie("%s"%avarie)
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.log_dispo()
            return 1
        if 9500 < random.randint(0, 10000 ) :
            self.nb_libre = self.nb_plot - self.nb_velos
            self.log_reparation()
            
        return 0

    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.log_dispo()
            return to_return

        return 0
    def get_nearest_station(self):
        x = self.position[0]
        y = self.position[1]
        deplacement_x = [-1, 0, 1]
        deplacement_y = [-1, 0, 1]
        deplacements  = [ (x_proche , y_proche) for x_proche in deplacement_x for y_proche in  deplacement_y if x_proche or y_proche]
        positions_proches = [(x +  a , y +  b) for a,b in  deplacements ]
        nearests = []
        for x2, y2 in positions_proches:
            if x2 + x < 0 or y2 + y < 0:
                continue 
            position_la_plus_proche = x2 * cote_len  + y2
            if 0 <= position_la_plus_proche and position_la_plus_proche < len(stations):
                nearests.append(stations[position_la_plus_proche])
        return nearests
        
                
    def __str__(self):
        return u"station %s : self.nb_libre = %s, self.nb_velos =%s "%(self.nom,self.nb_libre, self.nb_velos)


# 0) get_names

In [224]:
def get_names(nb_lettres=5, nb_noms = 15000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        nb_noms = nb_noms -1
        if nb_noms <2:
            break
    shuffle(names)
    return names

# 0) création des stations

In [225]:
import math

In [226]:
deplacement_x = [-1, 0, 1]
deplacement_y = [-1, 0, 1]

In [227]:
from time import time
import multiprocessing
nb_cpu       = 4
nb_station   = 40
debut        = time()
process_pool = multiprocessing.Pool(nb_cpu)
debut        = time()
names        = get_names()
cote_len     = int(math.ceil(math.sqrt(nb_station)))
positions    = [ (longitude, lattitude) for longitude in range(cote_len) for lattitude in range(cote_len)]
parameters   = [(names[i], positions[i]) for i in range(nb_station)]
stations     = process_pool.map(Station, parameters)
for x in stations:
    x.nearest = x.get_nearest_station()
nb_velos     = sum([len(station.velos) for station in stations])
duree = time() - debut
print "duree = %s pour %s cree soit %s sec / unite"%(duree, nb_velos, duree/nb_velos)


duree = 0.771585941315 pour 755 cree soit 0.00102196813419 sec / unite


# Creation des cycliste

In [228]:
aller_a = []
probas  = {}
n = -1
for station in stations:
    n = n + 1
    milieu      =  cote_len / 2 
    centre      = (milieu, milieu)
    adjacent_1  = (station.position[0] - milieu)
    adjacent_2  = (station.position[1] - milieu)
    hypothenuse = math.sqrt( adjacent_1*adjacent_1 + adjacent_2 *adjacent_2 ) 
    probas      = int(math.ceil(milieu - math.floor(hypothenuse) + 2))
    aller_a.extend([station]*probas)    

In [229]:
n = 4
process_pool = multiprocessing.Pool(n)
names        = get_names(5, nb_velos*1.2)
noms         = [(names[i], random.choice(stations), random.choice(aller_a)) for i in range(nb_velos)]
del names
cyclistes   = process_pool.map(Cycliste, noms)
print "duree = %s pour %s cree soit %s sec / unite"%(duree, len(cyclistes), duree/nb_velos)


duree = 0.771585941315 pour 755 cree soit 0.00102196813419 sec / unite


# voyages

In [230]:
from time import time
import multiprocessing
import math
from time import time

In [231]:
!rm ./logs/*

In [232]:
depart = time()
n = 0  
while time() < depart + 3600 : 
    a         = time()
    old_nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
    en_route = filter(lambda cycliste:     cycliste.sur_velo, cyclistes)
    a_pied   = filter(lambda cycliste: not cycliste.sur_velo, cyclistes)
    map(lambda cycliste: cycliste.rendre_velo(random.choice(stations)), en_route  )
    map(lambda cycliste: cycliste.prend_velo(random.choice(stations)),  a_pied    )
    new_nb_trajet =  sum([cycliste.nb_trajet for cycliste in cyclistes]) - old_nb_trajet
    duree_totale = time()-a
    print "(%03d)%03d/% 3.2f|"%(n, new_nb_trajet, duree_totale),

    n += 1
print "*"*30
print "fini"
print "*"*30

(000)000/ 0.76| (001)000/ 0.08| (002)000/ 0.09| (003)000/ 0.11| (004)000/ 0.13| (005)000/ 0.14| (006)000/ 0.16| (007)000/ 0.17| (008)000/ 0.18| (009)000/ 0.19| (010)000/ 0.19| (011)000/ 0.21| (012)000/ 0.21| (013)000/ 0.29| (014)000/ 0.25| (015)000/ 0.29| (016)000/ 0.26| (017)000/ 0.33| (018)000/ 0.27| (019)000/ 0.28| (020)000/ 0.29| (021)000/ 0.26| (022)000/ 0.28| (023)000/ 0.29| (024)000/ 0.28| (025)000/ 0.28| (026)000/ 0.27| (027)000/ 0.28| (028)000/ 0.28| (029)000/ 0.30| (030)000/ 0.31| (031)000/ 0.31| (032)000/ 0.33| (033)000/ 0.34| (034)000/ 0.33| (035)000/ 0.32| (036)000/ 0.33| (037)000/ 0.33| (038)000/ 0.34| (039)000/ 0.35| (040)000/ 0.36| (041)000/ 0.38| (042)000/ 0.37| (043)000/ 0.38| (044)000/ 0.37| (045)000/ 0.38| (046)000/ 0.37| (047)000/ 0.39| (048)000/ 0.42| (049)000/ 0.41| (050)000/ 0.45| (051)000/ 0.44| (052)000/ 0.44| (053)000/ 0.44|

KeyboardInterrupt: 